In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and 

In [2]:
!pip install datasets
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=0d1ebdfbb07f4a114f9272d556aa32b3d3962af9acc9d0c5f33590ccbd05e751
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [3]:
pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import json
import torch
from transformers import AutoTokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader

# Load your JSON data
with open("/content/foodreportdata.txt", "r") as f:
    data = json.load(f)

# Custom Dataset for T5
class FoodWasteDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        query = self.data[idx]["query"]
        output = self.data[idx]["query_output"]

        # Construct the input text
        input_text = f"Fooditem: {query['fooditem']}, Area of food wasted: {query['area_of_food_wasted']},area_of_plate_total_area:{query['area_of_plate_total_area']}, Percentage of food wasted: {query['percentage_of_food_wasted']}, Time of meal: {query['time_of_meal']}, Age: {query['age_of_person']}, Place: {query['place_where_food_was_eaten']}"

        # Construct the target text
        target_text = f"Consumer solution: {output['consumer_solution']} Management solution: {output['management_solution']}"

        # Tokenize the input and target text
        input_tokens = self.tokenizer(input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        target_tokens = self.tokenizer(target_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")

        # Prepare the input and target token IDs and attention masks
        source_ids = input_tokens["input_ids"].squeeze()
        target_ids = target_tokens["input_ids"].squeeze()

        source_mask = input_tokens["attention_mask"].squeeze()
        target_mask = target_tokens["attention_mask"].squeeze()

        return {
            "input_ids": source_ids,
            "attention_mask": source_mask,
            "labels": target_ids,
            "decoder_attention_mask": target_mask
        }

# Initialize the T5 tokenizer
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")

# Create the dataset
dataset = FoodWasteDataset(data, tokenizer)

# DataLoader for batching
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:

from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
from transformers import T5ForConditionalGeneration, AdamW

# Load the pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 20  # You can increase this based on your dataset size
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()

        # Move batch to device (GPU/CPU)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # Loss calculation
        loss = outputs.loss
        epoch_loss += loss.item()

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader)}")


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1/20, Loss: 3.1899565183199368
Epoch 2/20, Loss: 1.08247978412188
Epoch 3/20, Loss: 0.6816151738166809
Epoch 4/20, Loss: 0.5110043975023123
Epoch 5/20, Loss: 0.43954198177044207
Epoch 6/20, Loss: 0.40554611957990205
Epoch 7/20, Loss: 0.3766224476007315
Epoch 8/20, Loss: 0.3524871583168323
Epoch 9/20, Loss: 0.3300952475804549
Epoch 10/20, Loss: 0.3128267824649811
Epoch 11/20, Loss: 0.29702479105729324
Epoch 12/20, Loss: 0.28298855974124026
Epoch 13/20, Loss: 0.2732714277047377
Epoch 14/20, Loss: 0.2628520176960872
Epoch 15/20, Loss: 0.25061579736379475
Epoch 16/20, Loss: 0.2388915362266394
Epoch 17/20, Loss: 0.23185325012757227
Epoch 18/20, Loss: 0.22047058664835417
Epoch 19/20, Loss: 0.21312606220061964
Epoch 20/20, Loss: 0.20466860899558434


In [10]:
model.save_pretrained("fine_tuned_t5_food_waste_model")
tokenizer.save_pretrained("fine_tuned_t5_food_waste_model")


('fine_tuned_t5_food_waste_model/tokenizer_config.json',
 'fine_tuned_t5_food_waste_model/special_tokens_map.json',
 'fine_tuned_t5_food_waste_model/spiece.model',
 'fine_tuned_t5_food_waste_model/added_tokens.json')

In [11]:

def generate_solution(query_input):
    model.eval()

    input_tokens = tokenizer(query_input, return_tensors="pt", max_length=512, truncation=True)
    input_ids = input_tokens["input_ids"].to(device)
    attention_mask = input_tokens["attention_mask"].to(device)

    output_tokens = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=100)
    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    return generated_text

test_query = "Fooditem: Paneer Tikka, Area of food wasted: 135000, area_of_plate_total_area: 500000,Percentage of food wasted: 27%, Time of meal: Dinner, Age: 38, Place: Work Cafeteria"
print(test_query)
print(generate_solution(test_query))


Fooditem: Paneer Tikka, Area of food wasted: 135000, area_of_plate_total_area: 500000,Percentage of food wasted: 27%, Time of meal: Dinner, Age: 38, Place: Work Cafeteria



In [12]:
import torch
from sklearn.metrics import f1_score, accuracy_score, recall_score

def validate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            # Extract inputs and labels from the batch
            inputs = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)
            decoder_input_ids = labels[:, :-1].contiguous()

            outputs = model(input_ids=inputs, decoder_input_ids=decoder_input_ids)
            logits = outputs.logits
            _, preds = torch.max(logits, 2)

            all_preds.extend(preds.cpu().numpy().flatten())
            all_labels.extend(labels[:, 1:].contiguous().cpu().numpy().flatten())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')

    # Print the metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print(f'Recall: {recall:.4f}')

validate_model(model, test_loader, device)


Accuracy: 0.9375
F1 Score: 0.9112
Recall: 0.9375


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
import pickle
with open('food_waste_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('food_waste_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

print("Model and tokenizer saved to pickle files.")

Model and tokenizer saved to pickle files.
